In [1]:
from  Controller import TrainingController_V2, ExplainingController_V2
from Parameters.PredictingParameters import PredictingParameters
from Parameters.TrainingParameters import TrainingParameters
from Parameters.Enums import SelectableLoss, SelectableModels, SelectableDatasets
import torch
import pyAgrum.lib.notebook as gnb
from IPython.core.display import display, HTML

/Users/jrhs/.pyenv/versions/3.9.4/envs/dl/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
model_path = "./SavedModels/0.8318_BPI2012_BaseLineLSTMModel_2021-05-10 04:35:30.266388" 
loss_fn = SelectableLoss.CrossEntropy
model=SelectableModels.BaseLineLSTMModel
dataset = SelectableDatasets.BPI2012

In [3]:
TrainingParameters.model = model
TrainingParameters.loss  = loss_fn
TrainingParameters.load_model_folder_path = model_path
TrainingParameters.dataset = dataset

In [4]:
trainer = TrainingController_V2()


| Running on cpu  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/A 

| Model loaded successfully from: ./SavedModels/0.8318_BPI2012_BaseLineLSTMModel_2021-05-10 04:35:30.266388  


<Figure size 1600x800 with 0 Axes>

In [5]:
########### Get example data from trainer ###########
print("Test set length: %d" %(len(trainer.test_dataset)))
index_from_test = 45

Test set length: 1308


In [6]:
example_trace = trainer.model.vocab.list_of_index_to_vocab(trainer.test_dataset[index_from_test]["trace"])

In [7]:
########## Setting up predicting parameters #########
PredictingParameters.load_model_folder_path = model_path
PredictingParameters.loss = loss_fn

In [8]:
explainer = ExplainingController_V2()


| Model loaded successfully from ./SavedModels/0.8318_BPI2012_BaseLineLSTMModel_2021-05-10 04:35:30.266388/model.pt 


In [9]:
df, data_predicted_list, bn, bn_html, inference, infoBN, markov_blanket_html = explainer.pm_predict_lindaBN_explain(example_trace)


| Selecting Greedy Hill Climbing Algorithm 
/Users/jrhs/Documents/GitHub/Apply-LINDA-BN-on-predicting-next-event/Utils/VocabDict.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sorted_seq_list = [torch.tensor(seq_list[idx])


In [10]:
last_step_inference = gnb.getInference(
            bn, evs={list(df.columns)[-1][0]: example_trace[-1]}, targets=list([ col[0] for col in df.columns.values]), size="70")

In [11]:
display(HTML(last_step_inference))